In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from censai import RIMUnet, PhysicalModel
from censai.data import NISGenerator
import os

In [19]:
kappa_checkpoints_dir = os.path.join(os.getenv("HOME"), "scratch/Censai/models/UnetRIM_21-06-02_15-11-15/kappa_checkpoints/")
source_checkpoints_dir = os.path.join(os.getenv("HOME"), "scratch/Censai/models/UnetRIM_21-06-02_15-11-15/source_checkpoints/")


phys = PhysicalModel(
    pixels=512,
    src_pixels=3,
    image_fov=20,
    kappa_fov=22.2,
    method="conv2d",
    noise_rms=1e-3,
    kappalog=True,
)

rim = RIMUnet(phys, 16, adam=True, kappalog=True, normalize=False, state_sizes=[32, 32, 128, 512],
                  **{"source": {"strides": 4},
                     "kappa": {"strides": 2}})

source_ckpt = tf.train.Checkpoint(net=rim.source_model)
source_checkpoint_manager = tf.train.CheckpointManager(source_ckpt, source_checkpoints_dir, max_to_keep=3)
kappa_ckpt = tf.train.Checkpoint(net=rim.kappa_model)
kappa_checkpoint_manager = tf.train.CheckpointManager(kappa_ckpt, kappa_checkpoints_dir, max_to_keep=3)

kappa_checkpoint_manager.checkpoint.restore(kappa_checkpoint_manager.latest_checkpoint)
source_checkpoint_manager.checkpoint.restore(source_checkpoint_manager.latest_checkpoint)

In [20]:
gen = NISGenerator(1, batch_size=1, pixels=512)

In [25]:
lens, source, kap = gen.generate_batch_rim()
source_pred, kap_pred = rim.forward_pass(lens)

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,1,1,16] vs. shape[1] = [1,0,0,16] [Op:ConcatV2] name: concat